Primeiramente, se importa bibliotecas utilizadas e se estabelece parâmetros

In [23]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML

plt.rcParams['figure.figsize'] = [18, 9]

Em seguida, as funções necessárias ao longo do projeto são implementadas nas seguintes células

In [2]:
#Extrai coluna de lista
def pega_coluna(col_num, dados):
    coluna = []
    for i in range (0, len(dados)):
        linha = data[i]
        coluna.append(linha[col_num])
    return coluna

In [28]:
#Escolhe vertices de regiao de interesse com base em dimensões de imagem
def regiao_de_interesse(imagem):
    altura = imagem.shape[0]
    largura = imagem.shape[1]
    regiao = [
        (0, altura),
        (math.floor(largura * 0.4), math.floor(altura * 0.6)),
        (math.floor(largura * 0.6), math.floor(altura * 0.6)),
        (largura, altura)
    ] #região trapezoidal
    return regiao

In [4]:
# Corta imagem de acordo com uma região entre pontos estabelecidos
def corta_regiao(img, regiao):
    # Define uma matriz vazia de máscara com as dimensões da imagem:
    mascara = np.zeros_like(img)
    mascara_cor = 255
    # Preenche dentro do polígono
    cv2.fillPoly(mascara, regiao, mascara_cor)
    # Retorna a imagem onde os pixels combinam
    imagem_cortada = cv2.bitwise_and(img, mascara)
    return imagem_cortada


In [5]:
# Desenha as linhas na imagem
def desenha_linhas(imagem, linhas, cor=[255, 0, 0], espessura=3):
      imagem_linha = np.zeros(
        (
          imagem.shape[0],
          imagem.shape[1],
          3
        ),
        dtype=np.uint8
      )
      imagem = np.copy(imagem)
      if linhas is None:
        return
      for linha in linhas:
        for x1, y1, x2, y2 in linha:
          cv2.line(imagem_linha, (x1, y1), (x2, y2), cor, espessura)
      img = cv2.addWeighted(imagem, 0.8, imagem_linha, 1.0, 0.0)
      return img

In [6]:
#Lê imagem, redimensionando para o padrão estabelecido
def le_imagem(nome):
    imagem = mpimg.imread('exemplo_imagens\%s.jpg' %nome)
    imagem = cv2.resize(imagem, (960,540))
    return imagem

In [7]:
#Mostra a imagem em questão
def mostra_imagem(imagem):
    plt.imshow(imagem)
    plt.show()

In [1]:
def detecta_faixas(imagem):

    #Filtra e retira cores
    imagem_filtrada = filtra_amarelo_branco(imagem)
    
    #Obtém bordas
    imagem_canny = cv2.Canny(imagem_filtrada, 100, 200)
    
    #Corta região de interesse
    imagem_cortada = corta_regiao(imagem_canny, np.array([regiao_de_interesse(imagem_canny)], np.int32))
    
    #Detecta faixas:
    lines = cv2.HoughLinesP(
        imagem_cortada,
        rho=6,
        theta=np.pi / 60,
        threshold=70,
        lines=np.array([]),
        minLineLength=20,
        maxLineGap=25
    )

    linha_esquerda_x = []
    linha_esquerda_y = []
    linha_direita_x = []
    linha_direita_y = []

    for linha in linhas:
        for x1, y1, x2, y2 in linha:
            inclinacao = (y2 - y1) / (x2 - x1)
            if math.fabs(inclinacao) < 0.5:
                continue
            if inclinacao <= 0:
                linha_esquerda_x.extend([x1, x2])
                linha_esquerda_y.extend([y1, y2])
            else:
                linha_direita_x.extend([x1, x2])
                linha_direita_y.extend([y1, y2])

    min_y = int(imagem.shape[0] * (3 / 5))
    max_y = int(imagem.shape[0])

    poly_esquerda = np.poly1d(np.polyfit(
        linha_esquerda_y,
        linha_esquerda_x,
        deg=1
    ))

    esquerda_x_comeco = int(poly_esquerda(max_y))
    esquerda_x_fim = int(poly_esquerda(min_y))
    poly_direita = np.poly1d(np.polyfit(
        linha_direita_y,
        linha_direita_x,
        deg=1
    ))
    direita_x_comeco = int(poly_direita(max_y))
    direita_x_fim = int(poly_direita(min_y))

    imagem_linha = desenha_linhas(
        imagem,
        [[
            [esquerda_x_comeco, max_y, esquerda_x_fim, min_y],
            [direita_x_comeco, max_y, direita_x_fim, min_y],
        ]],
        espessura=5,
    )
    return imagem_linha

In [4]:
#Aplica a pipeline e grava o video
def produz_video(nomevideo):
    saidavideo = 'resultados\%s_saida.mp4' %nomevideo
    video = VideoFileClip("exemplo_videos\%s.mp4" %nomevideo)
    videofinal = video.fl_image(detecta_faixas)
    videofinal.write_videofile(saidavideo, audio=False)    
    videofinal.reader.close()
    videofinal.audio.reader.close_proc()
    

In [3]:
#Mostra as intensidades de cada canal de cor da imagem em RGB, HSV e HLS
def mostra_cores(imagem):
    
    #Filtra RGB:

    #separa os canais
    r,g,b = cv2.split(imagem)
    #plota
    figura, eixo = plt.subplots(ncols=3, nrows=1)
    eixo[0].imshow(r, cmap="gray")
    eixo[0].set_title('Red')
    eixo[1].imshow(g, cmap="gray")
    eixo[1].set_title('Green')
    eixo[2].imshow(b, cmap="gray")
    eixo[2].set_title('Blue')
    figura.suptitle("RGB", fontsize=30, y=0.72)
    plt.show()
    
    #Filtra HSV:
    
    #converte RGB -> HSV
    imagemhsv = cv2.cvtColor(imagem,cv2.COLOR_RGB2HSV)
    #separa os canais
    h,s,v = cv2.split(imagemhsv)
    #plota
    figura, eixo = plt.subplots(ncols=3, nrows=1)
    eixo[0].imshow(h, cmap="gray")
    eixo[0].set_title('Hue')
    eixo[1].imshow(s, cmap="gray")
    eixo[1].set_title('Saturation')
    eixo[2].imshow(v, cmap="gray")
    eixo[2].set_title('Value')
    eixo[0].set_ylabel('HSV')
    figura.suptitle("HSV", fontsize=30, y=0.72)
    plt.show()
    
    #Filtra HSL:
    
    #converte RGB -> HLS
    imagemhls = cv2.cvtColor(imagem,cv2.COLOR_RGB2HLS)
    #separa os canais
    h,l,s = cv2.split(imagemhls)
    #plota
    figura, eixo = plt.subplots(ncols=3, nrows=1)
    eixo[0].imshow(h, cmap="gray")
    eixo[0].set_title('Hue')
    eixo[1].imshow(l, cmap="gray")
    eixo[1].set_title('Lightness')
    eixo[2].imshow(s, cmap="gray")
    eixo[2].set_title('Saturation')
    figura.suptitle("HSL", fontsize=30, y=0.72)
    plt.show()
    

In [5]:
#Mostra canais de cor de várias imagens do formato RGB em tons de cinza
def testa_rgb(banco):
    
    #Obtém tamanho do banco:
    tam = len(banco)
    
    #Plota tudo:
    
    figura, eixo = plt.subplots(ncols=3, nrows=1)
    figura.suptitle("RGB", fontsize=30, y=0.72)
    imagem = banco[0][0]
    r,g,b = cv2.split(imagem)
    eixo[0].imshow(r, cmap="gray")
    eixo[0].set_title('Vermelho')
    eixo[0].set_ylabel(banco[0][1],fontsize=16)
    eixo[1].imshow(g, cmap="gray")
    eixo[1].set_title('Verde')
    eixo[2].imshow(b, cmap="gray")
    eixo[2].set_title('Azul')
    plt.show()
    
    for i,nome in zip(range(tam-1),banco[1:]):
        figura, eixo = plt.subplots(ncols=3, nrows=1)
        imagem = banco[i+1][0]
        r,g,b = cv2.split(imagem)
        eixo[0].imshow(r, cmap="gray")
        eixo[0].set_ylabel(banco[i+1][1],fontsize=16)
        eixo[1].imshow(g, cmap="gray")
        eixo[2].imshow(b, cmap="gray")
        plt.show()
    

In [6]:
#Mostra canais de cor HSV de várias imagens do formato RGB em tons de cinza
def testa_hsv(banco):
    
    #Obtém tamanho do banco:
    tam = len(banco)
    
    #Plota tudo:
    
    figura, eixo = plt.subplots(ncols=3, nrows=1)
    figura.suptitle("HSV", fontsize=30, y=0.72)
    imagem = banco[0][0]
    imagemhsv = cv2.cvtColor(imagem,cv2.COLOR_RGB2HSV)
    h,s,v = cv2.split(imagemhsv)
    eixo[0].set_ylabel(banco[0][1],fontsize=16)
    eixo[0].imshow(imagem)
    eixo[0].set_title('Imagem completa')
    eixo[1].imshow(s, cmap="gray")
    eixo[1].set_title('Saturação')
    eixo[2].imshow(v, cmap="gray")
    eixo[2].set_title('Valor')
    plt.show()
    
    for i,nome in zip(range(tam-1),banco[1:]):
        figura, eixo = plt.subplots(ncols=3, nrows=1)
        imagem = banco[i+1][0]
        imagemhsv = cv2.cvtColor(imagem,cv2.COLOR_RGB2HSV)
        h,s,v = cv2.split(imagemhsv)
        eixo[0].set_ylabel(banco[i+1][1],fontsize=16)
        eixo[0].imshow(imagem)
        eixo[1].imshow(s, cmap="gray")
        eixo[2].imshow(v, cmap="gray")
        plt.show()
    

In [15]:
#Mostra canais de cor HlS de várias imagens do formato RGB em tons de cinza

def testa_hls(banco):
    
    #Obtém tamanho do banco:
    tam = len(banco)
    
    #Plota tudo:
    
    figura, eixo = plt.subplots(ncols=3, nrows=1)
    figura.suptitle("HLS", fontsize=30, y=0.72)
    imagem = banco[0][0]
    imagemhls = cv2.cvtColor(imagem,cv2.COLOR_RGB2HLS)
    h,l,s = cv2.split(imagemhls)
    eixo[0].set_ylabel(banco[0][1],fontsize=16)
    eixo[0].imshow(imagem)
    eixo[0].set_title('Imagem completa')
    eixo[2].imshow(l, cmap="gray")
    eixo[2].set_title('Luminosidade')
    eixo[1].imshow(s, cmap="gray")
    eixo[1].set_title('Saturação')
    plt.show()
    
    for i,nome in zip(range(tam-1),banco[1:]):
        figura, eixo = plt.subplots(ncols=3, nrows=1)
        imagem = banco[i+1][0]
        imagemhls = cv2.cvtColor(imagem,cv2.COLOR_RGB2HLS)
        h,l,s = cv2.split(imagemhls)
        eixo[0].set_ylabel(banco[i+1][1],fontsize=16)
        eixo[0].imshow(imagem)
        eixo[2].imshow(l, cmap="gray")
        eixo[1].imshow(s, cmap="gray")
        plt.show()

In [17]:
#Filtra as cores selecionadas para o realce de faixas

def filtra_amarelo_branco(imagem):
    convertido = cv2.cvtColor(imagem, cv2.COLOR_RGB2HLS)
    # white color mask
    inferior = np.uint8([  0, 200,   0])
    superior = np.uint8([255, 255, 255])
    mascara_branca = cv2.inRange(convertido, inferior, superior)
    # yellow color mask
    inferior = np.uint8([ 10,   0, 100])
    superior = np.uint8([ 50, 255, 255])
    mascara_amarela = cv2.inRange(convertido, inferior, superior)
    # combine the mask
    mascara = cv2.bitwise_or(mascara_branca, mascara_amarela)
    final = cv2.bitwise_and(imagem, imagem, mascara = mascara)
    return cv2.cvtColor(cv2.cvtColor(final, cv2.COLOR_HLS2RGB), cv2.COLOR_RGB2GRAY)

In [27]:
# Main
produz_video('faixa_direita_branca')
produz_video('faixa_esquerda_amarela')
produz_video('desafio')


[MoviePy] >>>> Building video resultados\faixa_direita_branca_saida.mp4
[MoviePy] Writing video resultados\faixa_direita_branca_saida.mp4


100%|█████████▉| 221/222 [00:08<00:00, 25.42it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: resultados\faixa_direita_branca_saida.mp4 

[MoviePy] >>>> Building video resultados\faixa_esquerda_amarela_saida.mp4
[MoviePy] Writing video resultados\faixa_esquerda_amarela_saida.mp4


100%|█████████▉| 681/682 [00:32<00:00, 20.99it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: resultados\faixa_esquerda_amarela_saida.mp4 

[MoviePy] >>>> Building video resultados\desafio_saida.mp4
[MoviePy] Writing video resultados\desafio_saida.mp4


100%|██████████| 251/251 [00:24<00:00, 10.43it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: resultados\desafio_saida.mp4 

